In [99]:
# import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os


In [93]:
udp_2017 = pd.read_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/reference/udp_2017results.csv')
# display(udp_2017.head())
rec1_vars = pd.read_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/regression/all_vars_rec1.csv')
# display(rec1_vars.head())



#### Get Loan Data

In [175]:
parsed_data_path = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/parsed_data_1/'
files = [f for f in os.listdir(parsed_data_path) if f[-3:] == "csv" and f.split('_')[1] in ["2015", "2016", "2017"]]
dfs = []

for file in files:
    df_lol = []
    df = pd.read_csv(parsed_data_path+file)[["Geoid","Year", "type", "# HI borrower, LI tract", "# HI borrower, HI tract", 
                 "# LI borrower, LI tract", "# LI borrower, HI tract"]]
    dfs.append(df)
    all_years = pd.concat(dfs, axis=0)
    for tract, tract_df in all_years.groupby("Geoid"):
        total_loans = (tract_df["# HI borrower, LI tract"].sum()+ tract_df["# HI borrower, HI tract"].sum()
        +tract_df["# LI borrower, LI tract"].sum()+ tract_df["# LI borrower, HI tract"].sum())
        pct_HIB = (tract_df["# HI borrower, LI tract"].sum()+tract_df["# HI borrower, HI tract"].sum())/total_loans
        row_list = [tract, total_loans ,pct_HIB]

        df_lol.append(row_list)
        
                
all_20151617 = pd.DataFrame(df_lol, columns=["Geoid",  "Total loans 2015-2017", "% Loans to MHI borrowers"])
all_20151617.head(30)
    


files = [f for f in os.listdir(parsed_data_path) if f[-3:] == "csv" and f.split('_')[1] in ["2013", "2014"]]
dfs = []
for file in files:
    df_lol = []

    df = pd.read_csv(parsed_data_path+file)[["Geoid","Year", "type", "# HI borrower, LI tract", "# HI borrower, HI tract", 
                 "# LI borrower, LI tract", "# LI borrower, HI tract"]]
    dfs.append(df)
    all_years = pd.concat(dfs, axis=0)
    for tract, tract_df in all_years.groupby("Geoid"):
        total_loans = (tract_df["# HI borrower, LI tract"].sum()+ tract_df["# HI borrower, HI tract"].sum()
        +tract_df["# LI borrower, LI tract"].sum()+ tract_df["# LI borrower, HI tract"].sum())
        pct_HIB = (tract_df["# HI borrower, LI tract"].sum()+tract_df["# HI borrower, HI tract"].sum())/total_loans
        row_list = [tract, total_loans ,pct_HIB]
        df_lol.append(row_list)
                
all_201314 = pd.DataFrame(df_lol, columns=["Geoid",  "Total loans 2013-2014", "% Loans to MHI borrowers"])

all_201314.head()


    
files = [f for f in os.listdir(parsed_data_path) if f[-3:] == "csv" and f.split('_')[1] in ["2009"]]
df_lol = []
dfs = []
for file in files:
    df_lol = []
    df = pd.read_csv(parsed_data_path+file)[["Geoid","Year", "type", "# HI borrower, LI tract", "# HI borrower, HI tract", 
                 "# LI borrower, LI tract", "# LI borrower, HI tract"]]
    dfs.append(df)
    all_years = pd.concat(dfs, axis=0)
    for tract, tract_df in all_years.groupby("Geoid"):
        total_loans = (tract_df["# HI borrower, LI tract"].sum()+ tract_df["# HI borrower, HI tract"].sum()
        +tract_df["# LI borrower, LI tract"].sum()+ tract_df["# LI borrower, HI tract"].sum())
        pct_HIB = (tract_df["# HI borrower, LI tract"].sum()+tract_df["# HI borrower, HI tract"].sum())/total_loans
        row_list = [tract, total_loans ,pct_HIB]
        df_lol.append(row_list)
                
all_2009 = pd.DataFrame(df_lol, columns=["Geoid",  "Total loans 2009", "% Loans to MHI borrowers"])
all_2009.head()
print(len(all_2009))


1378


In [177]:
#these are duplicates
len(all_201314[all_201314["Geoid"].duplicated(keep='first')])
#1540 duplicates. if I get rid of these the numbers will match better
len(all_20151617[all_20151617["Geoid"].duplicated(keep='first')])
len(all_2009[all_2009["Geoid"].duplicated(keep='first')])




0

In [156]:
def encode_udp(row):
    if (row == 'LI - Ongoing Gentrification and/or Displacement'):
        return '6'
    if (row == 'LI - At Risk of Gentrification and/or Displacement'):
        return '4'    
    elif ('MHI - At Risk of Exclusion'):
        return '3'
    elif ('LI - Not Losing Low Income Households'):
        return '2'
    else:
        return '0'


In [157]:
rec1_udp_merge["UDP code"] = rec1_udp_merge["Typology"].apply(lambda x: encode_udp(x))
rec1_vars_udp =  rec1_udp_merge.drop(["geo_fips","Typology", "old disp type"], axis=1)
rec1_vars_udp.head()

,Geoid,Percent of population 25 years and over with Bachelor's degree,% below poverty level,% Single Family,% Owner Occupied,Percent NH White alone,Percent NH Black or African African alone,Percent NH Asian alone,Percent Hispanic,Total number of housing units,Median home value,Median income,Total Loans,UDP code
0,6001400100,0.345,3.1,0.912439,0.895260,0.769150,0.041435,0.110724,0.007312,1439,1000000.0,186439.0,68.0,3
1,6001400200,0.376,0.0,0.665236,0.657428,0.767823,0.022640,0.069364,0.106936,932,909500.0,122647.0,18.0,3
2,6001400300,0.321,6.9,0.478758,0.405179,0.714142,0.099114,0.075544,0.085214,2801,718100.0,66638.0,50.0,3
3,6001400400,0.440,4.0,0.555446,0.433809,0.698082,0.092588,0.078143,0.062988,2020,790500.0,80391.0,36.0,3
4,6001400500,0.281,6.0,0.444380,0.437722,0.419526,0.346635,0.066241,0.115988,1735,572000.0,50658.0,49.0,6


#### Merging Data for 2013-2014 model

In [158]:
rec1_udp_merge = pd.merge(rec1_vars, udp_2017, how='left', right_on='geo_fips', left_on='Geoid')
rec1_udp_loans_merge = pd.merge(rec1_udp_merge, all_201314, how='left', on='Geoid')
print("rec1vars: ", len(rec1_vars))
print("udp: ", len(udp_2017))
print("loans tract: ", len(all_201314)) #this has all the tracts not just the ones with neighbors. also can't use this
#as is because it has one row for each year, and I need to compute new proportions using the totals for the periods
print("merged 1: ", len(rec1_udp_merge))
print("merged 2: ", len(rec1_udp_loans_merge))


rec1vars:  1495
udp:  2131
loans tract:  3089
merged 1:  1495
merged 2:  2957


In [119]:
rec1_udp_loans_merge.head(20)
rec1_udp_merge.head(20)
all_201314.head(20)

,Geoid,Year,type,"# HI borrower, LI tract","# HI borrower, HI tract","# LI borrower, LI tract","# LI borrower, HI tract","% hib, lit","% lib, lit",% hib
0,6085500100,2013,mod,71,0,23,0,0.755319,0.244681,0.755319
1,6085500200,2013,mod,46,0,7,0,0.867925,0.132075,0.867925
2,6085500300,2013,mid,0,69,0,10,0.000000,0.000000,0.873418
3,6085500400,2013,mod,43,0,3,0,0.934783,0.065217,0.934783
4,6085500500,2013,high,0,81,0,7,0.000000,0.000000,0.920455
5,6085500600,2013,mid,0,48,0,7,0.000000,0.000000,0.872727
6,6085500800,2013,low,146,0,15,0,0.906832,0.093168,0.906832
7,6085500901,2013,mod,79,0,1,0,0.987500,0.012500,0.987500
8,6085500902,2013,low,3,0,0,0,1.000000,0.000000,1.000000
9,6085501000,2013,mod,40,0,3,0,0.930233,0.069767,0.930233
